<a href="https://colab.research.google.com/github/mu06905/GPU-Accelerated-Programming-in-Cuda-2023/blob/main/Week2/Class_Task_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g7u4tuxd
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g7u4tuxd
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=266641acebdfb786f1fd1e2cef58442d67924d0997ca4abca28c1769c51e3006
  Stored in directory: /tmp/pip-ephem-wheel-cache-eb3oxqcw/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}

//kernel function definition
__global__ void add(int *a , int *b, int *c,char *op) 
{
    if(*op=='+')
        *c = *a + *b;
    else if(*op=='-')
        *c = *a - *b;
    else if(*op == '*')
          *c = *a * *b;
    else if(*op =='/')
          *c = *a / *b;
}

int main() {
  int h_a = 10;
  int h_b = 20;
  int h_c = 10+20;
  char h_op = '-';
  int *d_a, *d_b, *d_c;
  char *d_op;

  //allocate memory on device
  cudaMalloc((void **)&d_a, sizeof(int));
  cudaMalloc((void **)&d_b, sizeof(int));
  cudaMalloc((void **)&d_c, sizeof(int));
  cudaMalloc((void **)&d_op, sizeof(char));

  //copy host data to device memory
  cudaMemcpy(d_a, &h_a, sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &h_b, sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_op,&h_op, sizeof(char), cudaMemcpyHostToDevice);
  
  //call kernel
  add<<<1,1>>>(d_a, d_b, d_c,d_op); 

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  checkCudaErr(cudaGetLastError(), "GPU");
  printf("Answer (on host): %d\n", h_c);

  //copy device data to host memory
  cudaMemcpy(&h_c, d_c, sizeof(int), cudaMemcpyDeviceToHost);
  printf("Answer (on device): %d", h_c);

  //release GPU memory
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  
  return 0;
}

Answer (on host): 30
Answer (on device): -10
